In [1]:
from datasets import load_dataset

# Charger le dataset
dataset = load_dataset("cbasu/Med-EASi")

# T5 fonctionne mieux avec une instruction explicite
prefix = "simplify: "

# Préparer les colonnes (source = Expert, target = Simple)
def preprocess(example):
    return {
        "input_text": prefix + example["Expert"],
        "target_text": example["Simple"]
    }

dataset = dataset.map(preprocess)

# Train/test split
train_dataset = dataset["train"].train_test_split(test_size=0.1, seed=42)
train_data = train_dataset["train"]
eval_data = train_dataset["test"]


In [2]:
from transformers import T5Tokenizer

tokenizer = T5Tokenizer.from_pretrained("t5-small")

# Paramètres max
MAX_INPUT = 512
MAX_TARGET = 128

def tokenize_function(example):
    model_inputs = tokenizer(
        example["input_text"], max_length=MAX_INPUT, padding="max_length", truncation=True
    )
    labels = tokenizer(
        example["target_text"], max_length=MAX_TARGET, padding="max_length", truncation=True
    )
    model_inputs["labels"] = labels["input_ids"]
    return model_inputs

train_tokenized = train_data.map(tokenize_function, batched=True)
eval_tokenized = eval_data.map(tokenize_function, batched=True)


You are using the default legacy behaviour of the <class 'transformers.models.t5.tokenization_t5.T5Tokenizer'>. This is expected, and simply means that the `legacy` (previous) behavior will be used so nothing changes for you. If you want to use the new behaviour, set `legacy=False`. This should only be set if you understand what it means, and thoroughly read the reason why this was added as explained in https://github.com/huggingface/transformers/pull/24565


In [4]:
from transformers import T5ForConditionalGeneration, Seq2SeqTrainer, Seq2SeqTrainingArguments, DataCollatorForSeq2Seq

model = T5ForConditionalGeneration.from_pretrained("t5-small")

training_args = Seq2SeqTrainingArguments(
    output_dir="./t5-med-simplify",
    save_strategy="epoch",
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    num_train_epochs=5,
    learning_rate=5e-5,
    weight_decay=0.01,
    predict_with_generate=True,
    logging_dir="./logs",
    logging_steps=50,
    save_total_limit=2,
    fp16=True,
    report_to="none"  # évite les erreurs si wandb n'est pas installé
)

data_collator = DataCollatorForSeq2Seq(tokenizer, model=model)

trainer = Seq2SeqTrainer(
    model=model,
    args=training_args,
    train_dataset=train_tokenized,
    eval_dataset=eval_tokenized,
    tokenizer=tokenizer,
    data_collator=data_collator,
)


/tmp/ipykernel_1128177/2100451326.py:23: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Seq2SeqTrainer.__init__`. Use `processing_class` instead.
  trainer = Seq2SeqTrainer(


In [ ]:
trainer.train()

/home/amadou/Documents/Project/MedSimplify-1/medisimpenv/lib/python3.12/site-packages/torch/utils/data/dataloader.py:665: UserWarning: 'pin_memory' argument is set as true but no accelerator is found, then device pinned memory won't be used.
  warnings.warn(warn_msg)
Passing a tuple of `past_key_values` is deprecated and will be removed in Transformers v4.48.0. You should pass an instance of `EncoderDecoderCache` instead, e.g. `past_key_values=EncoderDecoderCache.from_legacy_cache(past_key_values)`.


Step,Training Loss
